Author: Hasim Engin <br>
email:hengin@ciesin.columbia.edu<br>
Project: GRID3<br>
Organization:CIESIN, Columbia University<br>

# Create Catchments for a PoI 

This notebook creates accessibility surface and catchments for each point of interest in a PoI layer.
## Input layers:

* **Friction layers**: Friction layers (walk and walk+motorized) in minutes.They can be created by running "create_friction_surface" script.
* **PoI layer**: A point of interest layer. Such as health facilities, schools, water sources ...
* **Population raster layer**: It should be in the same resolution with friction layers (100m). Building foorprint constrained population layers that WorldPop produces are suggested. They can be downloaded here:https://www.worldpop.org/project/categories?id=3
* **Settlement extent layer**: This is a vector layer produced by GRID3 for sub-saharas countries. They can be downloaded here:https://grid3.org/resources/data
* **Admin boundary**:  Geographic extent of area. It can be a country extent , a province or a district. It will be used to determine outher boundary of catchments. In other words, the catchments will be nested into specified  admin boundaries. 

## Output layers: 

Output layers will be saved into a directory that is created specified workspace directory.Two gdb will be created in the directory to save output layers
 <br>
* **output.gbd** : The scripts run trought each specified admin units and creates output layers.It loops trought admin unit to nest output layers into admin units. Each admin units will have these layers ( "..." is place holder for name conventions that area specified in the script):<br>
   ... _accessibility_... : Accessibility raster  layer in minute per a meter.<br>
   ..._catchments_... :Catchments boundary<br>
   ..._Sett_extent_cost_in_min_... : Settlement extent with distance to the closest PoI in minutes<br>
   ..._sett_path_... : The shorthest path from a settlement to the closest PoI<br>
   ..._sett_path_smoothlines_... : Smoothed version of the shorthest path<br>
 <br>
* **final.gbd**: All indivual admin specific admin outputs will be merged and saved in this gdb.These layers will be created in the final.gdb:<br>
   ..._based_accessibility_in_min: Accessibility raster  layer in minute per a meter.<br>
   ..._based_catchments :Catchments boundary<br>
   ..."_based_settlements_in_min : Settlement extent with distance to the closest PoI in minutes<br>
   ..."_based_settlement_shortest_path : Smoothed version of the shorthest path<br>
   <br>
Note: Catchments can be created at a county extent ( without nesting into admin units). When this is the case, "final.gdb" will be empty. Output layers in the "output.gdb" should be used.    
    
    


## import Libraries

In [ ]:

# import libraries
import os
import arcpy
import re
import unidecode
import pandas as pd
import arcpy.cartography as CA
import numpy as np
from arcgis.features import SpatialDataFrame
arcpy.env.overwriteOutput = True


In [ ]:
# required libraries can be installed with following code
#uncommaent the line below and write a -library name- after !):
#! -library name- install

## User Inputs

In [ ]:

###=================== Initialize Variables ===================###

country_iso=""       # three letter country iso code e.g. zmb for Zombia
poi_type=  ""           # type of point of interest such as "health_facility"      !! no space between names !!

travel_type= ""       # options: walk or mix( walk+motorized)
poi_subtype=  ""      # subtype of PoI  type. If there is no subtype , use "all"
admin_col=""         # field/column that hold admin names in admin boundary layer.
                          # if you want to create output layers without nesting them into
                            # a admin unit, create a dummy variable such as country iso code 
                            # then specify admin_col variale with the dummy variable

###================================ input layers ==============================================###

# path to PoI layer such as health facilities or schools
path_to_poi=r""

# friction surface for walking. It is created by "create_friction_surface" script
path_to_friction_walk=r""

# friction surface for motorized.It is created by "create_friction_surface" script
path_to_friction_mix=r""

#path to population raster
path_to_pop_raster=r"" # WorldPop building footprint contraint population layer

#path to admin  boundary
path_to_pop_admin_boundary=r"" 

# path to settlement extent
path_to_sett_extent=r""
### important: Settlement extent layers (bua, saa , hamlets) need to be merged. 

# path to the directory that you want to save outputs
output_path=r""

In [ ]:


###==================================== functions =============================================###

def project_raster(raster_to_be_projected, reference_raster):
    '''project a raster layer'''
    arcpy.env.snapRaster  =reference_raster
    arcpy.env.extent      =reference_raster
    arcpy.env.cellAlignment = "ALIGN_WITH_PROCESSING_EXTENT"
    arcpy.env.cellSize = reference_raster
    arcpy.env.overwriteOutput = True 
    arcpy.env.outputCoordinateSystem = arcpy.Describe(reference_raster).spatialReference
    prj= arcpy.Describe(reference_raster).spatialReference
    arcpy.ProjectRaster_management(raster_to_be_projected,"pop_proj",prj)
    arcpy.env.workspace = output_gdb


def spatial_join(target, join, fieldname, out_name):
    '''spatial join two layers by taking mean of target variable'''
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(target)
    fieldmappings.addTable(join)
    fieldIndex_ = fieldmappings.findFieldMapIndex("grid_code")
    fieldmap = fieldmappings.getFieldMap(fieldIndex_ )
    field = fieldmap.outputField
    field.name = fieldname
    field.aliasName = fieldname
    fieldmap.outputField = field
    fieldmap.mergeRule = "mean"
    fieldmappings.replaceFieldMap(fieldIndex_ , fieldmap)
    arcpy.SpatialJoin_analysis(target, join,out_name, "#", "#", fieldmappings)


def updated_boundary_to_admin(boundaryToBeAdjusted, admin_boundary, out_name):
    '''nest catchments into admins boundary'''
    smooth_boundaries=os.path.join(output_gdb,"smooth_boundaries")
    clip_by_admin=os.path.join(output_gdb,"clip_by_admin")
    union_with_admin=os.path.join(output_gdb,"union_by_admin")
    multipart=os.path.join(output_gdb,"multipart")
    neighbors_table=os.path.join(output_gdb,"neighbors_table")
    neighbors_table_npArray=os.path.join(output_gdb,"neighbors_table_npArray")
    neighbors_table_df=os.path.join(output_gdb,"neighbors_table_df")
    multipart_npArray=os.path.join(output_gdb,"multipart_npArray")
    multipart_df=os.path.join(output_gdb,"multipart_df")
    multipart_temp=os.path.join(output_gdb,"multipart_temp")
    multipart_final=os.path.join(output_gdb,out_name)
    
    ##======================================================================##
    arcpy.cartography.SmoothPolygon(boundaryToBeAdjusted,smooth_boundaries,
                                    "PAEK", "200 Meters", "FIXED_ENDPOINT", "RESOLVE_ERRORS", None)
    
    arcpy.Clip_analysis(smooth_boundaries,admin_boundary,clip_by_admin)
    arcpy.Union_analysis([clip_by_admin,admin_boundary],union_with_admin)
    arcpy.MultipartToSinglepart_management(union_with_admin,multipart)
    
    arcpy.AddField_management(multipart,"unique_id", "TEXT")
    arcpy.CalculateField_management(multipart,"uniqueid",'str(!OBJECTID!)+"_"+str(!gridcode!)', "PYTHON_9.3","")
    
    arcpy.PolygonNeighbors_analysis(multipart,neighbors_table, ["uniqueid"],"",
                                       "BOTH_SIDES","","METERS","SQUARE_KILOMETERS")
    neighbors_table_npArray =arcpy.da.TableToNumPyArray(neighbors_table,"*")
    neighbors_table_df=pd.DataFrame(neighbors_table_npArray)
    
    arcpy.AddField_management(multipart,"area_km", "DOUBLE")
    arcpy.CalculateGeometryAttributes_management(multipart, [ ["area_km", "AREA_GEODESIC"]],
                                                 "","SQUARE_KILOMETERS")
    
    multipart_npArray=arcpy.da.FeatureClassToNumPyArray( multipart,["uniqueid","area_km"])
    multipart_df=pd.DataFrame(multipart_npArray)
    merge_tables=pd.merge(neighbors_table_df,multipart_df,left_on="src_uniqueid", right_on="uniqueid")
    
    merge_tables=merge_tables[merge_tables["area_km"]<=0.1]
   
    merge_tables["gridcode2"]=np.nan
    get_unique_id=list(set(merge_tables.src_uniqueid.tolist()))

    
    for i in get_unique_id:
        i_check=i.split("_")[1:]
       
        rows=merge_tables.loc[merge_tables["src_uniqueid"]==i]
        rows_=rows[~rows["nbr_uniqueid"].str.split("_", expand=True)[1].isin(i_check)]
        if rows_.shape[0]>=1:
            get_max_lenght=rows_.LENGTH.idxmax()
            get_max_lenght_class=rows.loc[get_max_lenght,"nbr_uniqueid"]
            row_index=list(rows.index.values)
            merge_tables.loc[row_index,'gridcode2_']=get_max_lenght_class
         
    merge_tables=merge_tables[~merge_tables["gridcode2_"].isnull()] 
    merge_tables.to_csv(output_loc+"\\merge_table.csv") 
    merge_tables[["var1","gridcode2"]] = merge_tables['gridcode2_'].str.split("_", expand=True)
    merge_tables=merge_tables[["src_uniqueid","nbr_uniqueid",'gridcode2']]
    sdf=pd.DataFrame.spatial.from_featureclass(multipart)[["gridcode","uniqueid","SHAPE"]]
    sdf_merge=pd.merge(sdf,merge_tables,left_on="uniqueid",right_on="src_uniqueid",how="left")
    sdf_merge.loc[sdf_merge['gridcode2'].isnull(),'gridcode2'] = sdf_merge['gridcode']   
    sdf_merge.spatial.to_featureclass(multipart_temp)
    arcpy.Dissolve_management(multipart_temp,multipart_final,[["gridcode2","uniqueid"]],
                              None, "SINGLE_PART", "DISSOLVE_LINES") 
    arcpy.AlterField_management(multipart_final, "gridcode2", 'gridcode',"gridcode")
    ##===========================================================================###
    
    delete_fc=[smooth_boundaries,clip_by_admin,union_with_admin,multipart,multipart_temp]
  
    for fc in delete_fc:
        arcpy.Delete_management(fc) 
    


def create_accessibility_layers2(admin_name):
    ''' creates catchments for a PoI layer''' 
    
    if not arcpy.Exists(admin_name+"_accessibility_"+poi_subtype):
        
        ## initiliaze admin specific temporal layers
        admin_boundary=admin_name+"_boundary"
        admin_poi=admin_name+"_poi"
        admin_sett=admin_name+"_sett"
        admin_friction=admin_name+"_friction"
        
        ##==========================================================================================###
        ## initialize output layer names
        cost_distance=admin_name+"_accessibility_"+  poi_subtype
        catchment_areas_update= admin_name+"_catchments_"+  poi_subtype
        sett_path=admin_name+"_sett_path_"+  poi_subtype
        sett_path_smoothline=admin_name+"_sett_path_smoothlines_"+  poi_subtype
        sett_with_cost=admin_name+"_Sett_extent_cost_in_min_"+  poi_subtype

        ##==========================================================================================### 

        ## extract layers by the admin unit
        arcpy.Select_analysis(admins_bndry,admin_boundary,"admin_cln = '{}'".format(admin_name))
        arcpy.Select_analysis(poi,admin_poi,"admin_cln = '{}'".format(admin_name))
        arcpy.gp.ExtractByMask_sa(friction_layer,admin_boundary,admin_friction)
        arcpy.Clip_analysis(path_to_sett_extent, admin_boundary,admin_sett)
        
        if int(arcpy.GetCount_management(admin_poi).getOutput(0))>=1:


            print (f"Creating accessiblility layers for =={country_iso}== for =={poi_type}== with =={travel_type}== travel mode" )
            print(" ")
            print ( "   >>> Creating cost distance in minutes to PoI")
            arcpy.sa.CostDistance(admin_poi,admin_friction,"" , "outBkLinkRaster").save(  cost_distance)

            print ( "   >>> Creating catcment areas of PoI  based on  minimum cost")
            arcpy.sa.CostAllocation(admin_poi,admin_friction,"","","OBJECTID","cost_allocation").save("cost_allocation_temp")
            arcpy.RasterToPolygon_conversion("cost_allocation_temp", "catchment_areas_temp", "NO_SIMPLIFY")

            #------------------------------------------------------------------------------------------#    
            print ("   >>> Calculating population of each catchmet area by 30 min dist. interval")     
            arcpy.sa.Con(cost_distance,130, cost_distance, "VALUE>120").save(cost_distance+"_con")
            arcpy.sa.Contour(cost_distance+"_con", cost_distance+"_contour", 30, 0, 1, "CONTOUR_POLYGON")
            arcpy.AddField_management(cost_distance+"_contour", "time_interval", "TEXT")
            arcpy.management.CalculateField(cost_distance+"_contour", "time_interval", 'str(!ContourMin!)+"-"+str(!ContourMax!)', "PYTHON3", '', "TEXT")
            arcpy.Intersect_analysis([cost_distance+"_contour","catchment_areas_temp"], "catchment_areas_int", "ALL")
            arcpy.AddField_management( "catchment_areas_int", "zone_unique", "TEXT")
            arcpy.management.CalculateField( "catchment_areas_int", "zone_unique", 'str(!gridcode!)+"_"+str(!time_interval!)', "PYTHON3", '', "TEXT")
            arcpy.sa.ZonalStatisticsAsTable("catchment_areas_int", "zone_unique", path_to_pop_raster, "zonal_pop_min","DATA", "SUM")
            totpop_min_array = arcpy.da.TableToNumPyArray("zonal_pop_min",["zone_unique","SUM"])
            df=pd.DataFrame(totpop_min_array)
            df[['gridcode','time_interval']] = df[ "zone_unique"].str.split("_",expand=True)
            df= df.pivot(index='gridcode', columns='time_interval', values="SUM")
            df.rename({"0.0-30.0":"under_30min","30.0-60.0":"30min_60min","60.0-90.0":"60min_90min","90.0-120.0":"90min_120min",
                         "120.0-130.0":"above_120min"}, axis=1, inplace=True)
            df=   df.fillna(0)
            col_to_sum=[c for c in df.columns if  c.endswith("min")]
            df["totpop_20"]=df[col_to_sum].sum(axis=1)
            df_=pd.DataFrame(df.to_records())
            df_["gridcode"]=df_["gridcode"].astype(str).astype("int64")
           

            print ("   >>> Updating catchment boundary based on admin boundary")
            updated_boundary_to_admin("catchment_areas_temp", admin_boundary,catchment_areas_update)
            
            sdf=pd.DataFrame.spatial.from_featureclass(catchment_areas_update)
            sdf["gridcode"]=sdf["gridcode"].astype(str).astype("int64")
            sdf_poi=pd.DataFrame.spatial.from_featureclass(admin_poi)
            sdf_poi.drop(["SHAPE", 'NEAR_DIST',"NEAR_FID","admin_cln"], axis=1, inplace=True)
            sdf_poi.rename({"OBJECTID":"gridcode"},axis=1, inplace=True)
       
            
            sdf_merge1=sdf.merge(df_, how="left",on="gridcode")
            sdf_merge2=sdf_merge1.merge(sdf_poi, how="left",on="gridcode")
            numeric_columns = sdf_merge2.select_dtypes(include=['int64',"float64"]).columns
            sdf_merge2[numeric_columns ]=sdf_merge2[numeric_columns].fillna(0)
            numeric_columns = sdf_merge2.select_dtypes(include=['object']).columns
            sdf_merge2[numeric_columns ]=sdf_merge2[numeric_columns].fillna('')
            arcpy.Delete_management(catchment_areas_update)
            sdf_merge2.spatial.to_featureclass(output_gdb+"\\"+catchment_areas_update)

            #---------------------------------------------------------------------------------#

            print ("   >>> Creating cost path from settlements to PoI")
            arcpy.sa.CostPathAsPolyline(path_to_sett_extent, cost_distance, "outBkLinkRaster", sett_path, "EACH_ZONE", "OBJECTID", "INPUT_RANGE")

            print ("   >>> Smoothing paths")
            CA.SimplifyLine(sett_path,sett_path_smoothline, "POINT_REMOVE", 100,"","NO_KEEP")

            print ( "   >>> Calculating everage travel time in minutes to closest PoI...")
            arcpy.RasterToPoint_conversion(cost_distance, "cost_distance_point", "VALUE")
            spatial_join(admin_sett, "cost_distance_point", "average_cost_min", sett_with_cost)  
            arcpy.DeleteField_management( sett_with_cost,  ["Join_Count", "TARGET_FID", "pointid"])

            print ("   >>> Deleting temporary layers...")
            delete_fc=["outBkLinkRaster","cost_allocation_temp", "cost_distance_point","catchment_areas_int", "catchment_areas_temp","zonal_pop_min",
                       "settlement_path_temp","cost_allocation","cost_distance_all_con","cost_distance_all_contour",admin_name+"_boundary",
                       admin_name+"_poi",admin_name+"_poi",admin_name+"_friction",admin_name+"_sett",admin_name+"_accessibility_all_con",
                      admin_name+"_accessibility_all_contour" ]
            for fc in delete_fc:
                 arcpy.Delete_management(fc)      
            print ("##====================================================================================================##")    


def merge_outputs(select_layer_to_be_merged, output_name):
    '''merge admin specif output vector layers'''
    merge_fc=[fc for fc in arcpy.ListFeatureClasses() if fc.endswith(select_layer_to_be_merged)]
    if len(merge_fc):
        arcpy.Merge_management(merge_fc, final_gdb+"\\"+output_name)

def merge_raster(select_layer_to_be_merged, output_name):
    '''merge admin specif output raster layers'''
    merge_rasters=[fc for fc in arcpy.ListRasters() if fc.endswith("cost_distance_all")]
    if len(merge_rasters):
        arcpy.MosaicToNewRaster_management(merge_rasters, final_gdb, output_name,"","","",1)




## Process

In [ ]:


###==================== select friction layer based on travel type============================###

if travel_type=="walk":
    friction_layer= path_to_friction_walk
if travel_type=="mix":
    friction_layer= path_to_friction_mix
            
###================================ create output workspace ==========================================###

# create output directory
if not  os.path.exists (os.path.join(output_path,"output_"+poi_type+"_"+poi_subtype+"_"+travel_type)):
    os.mkdir(os.path.join(output_path,"output_"+poi_type+"_"+poi_subtype+"_"+travel_type))
output_loc=os.path.join(output_path,"output_"+poi_type+"_"+poi_subtype+"_"+travel_type)

# create output gdb
if  not arcpy.Exists(os.path.join(output_loc,"output.gdb")):
    arcpy.CreateFileGDB_management(output_loc,"output.gdb")
output_gdb=os.path.join(output_loc,"output.gdb")

if  not arcpy.Exists(os.path.join(output_loc,"final.gdb")):
    arcpy.CreateFileGDB_management(output_loc,"final.gdb")
final_gdb=os.path.join(output_loc,"final.gdb")

arcpy.env.workspace = output_gdb

###======================= copy admin units and PoI into workspace ==================================###

arcpy.CopyFeatures_management(path_to_poi, poi_type+"_"+poi_subtype)
poi=poi_type+"_"+poi_subtype
arcpy.CopyFeatures_management(path_to_pop_admin_boundary, "admin_boundary")
admins_bndry="admin_boundary"
poi="hf_all"



#==================================== format admin names =============================================###
objectid_list=[]
admin_list_=[]
arcpy.AddField_management(admins_bndry, "admin_cln", "TEXT")
with arcpy.da.UpdateCursor(admins_bndry,[admin_col,"admin_cln", 'OBJECTID']) as cursor:
    for row in cursor:
        if row[0] is not None:
            fix_admin= unidecode.unidecode(row[0])
            fix_admin=re.sub(r'[-|$|.|!|)|(|!|-|_|]',r'',fix_admin) # spatial characters need to be removed in admin names
            fix_admin=fix_admin.strip().replace("  ","_").replace(" ","_")
            row[1]=fix_admin
            admin_list_.append(row[1])
            objectid_list.append(row[2])
            cursor.updateRow(row)

            

admin_list2_=[]

 ##Transfer admin names from admin boundary to PoI layer.
 ##Admin names in boudary laeyr and PoI layer has to be identical
arcpy.Near_analysis(poi,admins_bndry)
arcpy.AddField_management(poi, "admin_cln", "TEXT")
for i, name_ in zip(objectid_list,admin_list_): 
    with arcpy.da.UpdateCursor(poi,["NEAR_FID","admin_cln"]) as cursor:
        for row in cursor:
            if row[0]==i:
                row[1]=name_
                admin_list2_.append(name_)
            cursor.updateRow(row)

admin_list=list(set(admin_list2_))

#==================================== project population raster ==================================###

project_raster(path_to_pop_raster, friction_layer)
path_to_pop_raster="pop_proj"


#==================================== main process =============================================###



print(f"There are {len(admin_list)} admin units to be processed..." )

## Create output by each admin units
for admin in admin_list:

    try:
        print ("##########  "+admin+"  ###########")
        create_accessibility_layers2(admin)
    except:
        print (admin+"  did NOT RUN")

#=========================== merge admin specific admin outputs ==========================###
("   >>> Merging district based output layers...")
merge_outputs("catchment_update_all",country_iso+"_"+poi_type+"_"+travel_type+"_based_catchments" )
merge_outputs("sett_path_smoothlines_all",country_iso+"_"+poi_type+"_"+travel_type+"_based_settlement_shortest_path" )
merge_outputs("Sett_extent_cost_in_min_all",country_iso+"_"+poi_type+"_"+travel_type+"_based_settlements_with_time" )
merge_raster("cost_distance_all", country_iso+"_"+poi_type+"_"+travel_type+"_based_access_surface_in_min")

print ("SCRIPT IS DONE")



